In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Prepare data

In [2]:
train_data = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv',index_col=0)
test_data = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv',index_col=0)

In [3]:
y = train_data['pressure']
X = train_data.drop(['pressure','breath_id'],axis=1)

In [4]:
X_test = test_data.drop(['breath_id'],axis=1)

In [5]:
STEPS = 80
N_FEATURES = X.shape[1]

In [6]:
y = y.values.reshape(-1,STEPS)
X = X.values.reshape(-1,STEPS,N_FEATURES)
X_test = X_test.values.reshape(-1,STEPS,N_FEATURES)

# Prepare model

In [11]:
from tensorflow import keras
from tensorflow.keras.layers import InputLayer,Dense,LSTM,Dropout,Flatten

In [15]:
mlp_tf = keras.Sequential([
        InputLayer(input_shape=(STEPS,N_FEATURES)),
        LSTM(200, return_sequences=True),
        LSTM(150, return_sequences=True),
        LSTM(100, return_sequences=True),
        Flatten(),
        Dense(100, activation="swish"),
        Dropout(0.2),
        Dense(150, activation="swish"),
        Dropout(0.2),
        Dense(200, activation="swish"),
        Dense(80, activation="relu")
    ])

In [16]:
callback_1 = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [17]:
mlp_tf.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss='mean_squared_error',metrics=['mse','mae'])

In [18]:
history = mlp_tf.fit(X, y, validation_split=0.2, epochs=30, batch_size=512, callbacks=[callback_1])

# Predict test pressures

In [19]:
y_pred_test = mlp_tf.predict(X_test)

In [22]:
plt.figure(figsize=(18,8))
loss=history.history['loss']
val_loss=history.history['val_loss']
epoch = range(1,len(loss)+1)
plt.plot(epoch, loss, 'r' , label="Training")
plt.plot(epoch, val_loss, 'b', label="Validation")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Training and validation loss')
plt.show()

In [23]:
output = pd.DataFrame({'id': test_data.index, 'pressure': y_pred_test.reshape(-1)})
output.to_csv('submission.csv', index=False)

In [24]:
output